# Programmable ROM and Forking Lemma

This section introduces the programmable random oracle model and the forking lemma, which are fundamental tools for analyzing cryptographic protocols in the random oracle model.

## Programmable Random Oracles

In the random oracle model, we can program the oracle to return specific values for certain inputs, as long as we maintain consistency and the correct distribution.

For example, consider the hash commitment scheme $\mathsf{HashCom}[\mathsf{H}]$ where commitments are of the form $\mathsf{H}(m \| r)$.
A reduction can program the oracle such that certain relations hold between committed messages, while maintaining the adversary's view.

The following proposition demonstrates the power of programming oracles by showing that despite a radical change in how commitments are generated, the games remain indistinguishable to any efficient adversary.
Specifically, $\mathsf{Game}_1$ forces the XOR of committed messages to be equal to a message $m^*$ that is determined before even running the adversary.
The game first draws $m^*$, runs the adversary with a random hash $h$ and obtains a hash $h'$ from the adversary.
If the adversary created $h'$ as $\mathsf{Commit}(m', r')$, then when the adversary is invoked a second time we have

$$
\begin{align*}
  \mathsf{Commit}(m, r) &= h\\
  \mathsf{Commit}(m', r') &= h'\\
  m \oplus m' &= m^*.
\end{align*}
$$

Note that the challenger's commitment $h$ was given to the adversary before the adversary returned commitment $h'$ and the adversary has never revealed the opening $(m', r')$ to the challenger.

A similar technique is used in the MuSig multi-signature scheme to simulate signing queries (i.e., to answer signing queries without knowledge of the secret key, as explained in the next section).

:::{note} Games: Programmable Random Oracle
:label: fig-prog-rom-games

**Game 0:**
```
params ← HashCom.Setup(1^λ)
T := ∅
m ←$ {0,1}
r ←$ {0,1}^λ
h := H(m || r)
(state, h') ← A^H(params, h)
return A^H(state, m, r)
```

**Oracle H(x):**
```
if T[x] = ⊥ then
    T[x] ←$ {0,1}^λ
return T[x]
```

**Game 1:**
```
params ← HashCom.Setup(1^λ)
T := ∅
m* ←$ {0,1}
h ←$ {0,1}^λ
(state, h') ← A^H(params, h)
if ∃(m', r') : T[m' || r'] = h' then
    Choose any such (m', r')
    m := m* ⊕ m'
else
    m ←$ {0, 1}
r ←$ {0,1}^λ
assert T[m || r] = ⊥
T[m || r] := h
return A^H(state, m, r)
```
:::

:::{important} Proposition: Programmable Random Oracle
:label: prop-prog-rom

Let $\mathsf{Game}_0$ and $\mathsf{Game}_1$ be as defined in {ref}`fig-prog-rom-games`.
For any adversary $\mathcal{A}$ making at most $q$ queries to the oracle $\mathcal{O}_H$, we have

$$
\left|\Pr[\mathsf{Game}_0 = 1] - \Pr[\mathsf{Game}_1 = 1]\right| \leq \frac{q}{2^\lambda}
$$
:::

The proof is left as an exercise (see {ref}`ex-prog-rom`).

## The Forking Lemma

The forking lemma is a fundamental tool for analyzing algorithms that interact with random oracles.
It considers an algorithm $\mathcal{A}$ with oracle access to $\mathcal{O}_H$, which is executed twice on different but related instances of the oracle:

- In the first execution, all random oracle responses are sampled uniformly as usual.
- In the second execution, the responses are identical to those from the first execution except for a specific query called the *forking point*, whose response is *refreshed* (resampled with fresh randomness).

As a result, the two executions of $\mathcal{A}$ proceed identically up to the forking point but may diverge afterwards.
When $\mathcal{A}$ succeeds, it outputs a tuple $(z, \mathit{aux})$, and the oracle query on input $z$ becomes the forking point.

The forking lemma bounds the probability that $\mathcal{A}$ succeeds in both executions and outputs the same value $z$, despite receiving different oracle responses for query $z$ in each execution.
Even though it appears that $\mathcal{A}$ can behave completely differently after receiving the refreshed oracle response, the forking lemma shows that $\mathcal{A}$ frequently still outputs the same $z$ value.
While the forking lemma appears very technical, it provides a powerful abstraction that we will invoke when proving the EUF-CMA security of Schnorr signatures.

:::{note} Games: Single and Fork
:label: fig-fork-games

**Game Single:**
```
inp ← InpGen(1^λ)
ρ ←$ R
T := ∅
α ← A^H(inp; ρ)
assert α ≠ ⊥
return 1
```

**Oracle H(x):**
```
if T[x] = ⊥ then
    T[x] ←$ S
return T[x]
```

**Game Fork:**
```
inp ← InpGen(1^λ)
ρ ←$ R
T := ∅
α ← A^H(inp; ρ)
assert α ≠ ⊥
(z, aux) := α
T' := T    // Copy table
T'[z] ←$ S
α' ← A^H'(inp; ρ)
assert α' ≠ ⊥
(z', aux') := α'
assert z = z' ∧ T[z] ≠ T'[z]
return (z, aux, z', aux')
```

**Oracle H'(x):**
```
if T'[x] = ⊥ then
    T'[x] ←$ S
return T'[x]
```
:::

:::{important} Lemma: Local Forking Lemma
:label: lem-fork

Let $q \geq 1$ be an integer and $\mathsf{InpGen}$ be a randomized algorithm that outputs some input $\mathit{inp}$.
Let $\mathcal{A}$ be a randomized algorithm that takes input $\mathit{inp}$ and randomness $\rho \in \mathcal{R}$, has oracle access to $\mathcal{O}_H: \{0,1\}^* \rightarrow S$, makes at most $q$ queries to $\mathcal{O}_H$, and returns either $\bot$ or a pair $(z, \mathit{aux})$ where $z \in \{0,1\}^*$ and $\mathit{aux}$ is auxiliary output.

For games $\mathsf{Game~Single}[\mathsf{InpGen}]$ and $\mathsf{Game~Fork}[\mathsf{InpGen}]$ defined in {ref}`fig-fork-games`, define:

$$
\begin{align*}
\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} &:= \Pr[\mathsf{Game~Single}[\mathsf{InpGen}] = 1] \\
\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} &:= \Pr[\mathsf{Game~Fork}[\mathsf{InpGen}] \neq 0]
\end{align*}
$$

Then:

$$
\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} \geq \mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} \left( \frac{\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}}}{q} - \frac{1}{|S|} \right)
$$
:::

**Reference:** This is the Local Forking Lemma by Bellare, Dai, and Li (Asiacrypt 2019).

## Exercises

:::{exercise}
:label: ex-prog-rom-games

Describe the similarities and differences between games $\mathsf{Game}_0$ and $\mathsf{Game}_1$ in {ref}`fig-prog-rom-games`.

:::{dropdown} **Solution**

In both games:
- The oracle $\mathcal{O}_H$ behaves identically as a random oracle
- The adversary receives a commitment $h$ and later the opening $(m, r)$
- The games return the same bit $b$ output by the adversary

The key differences are:
- In $\mathsf{Game}_0$: $m$ and $r$ are chosen first, then $h = \mathcal{O}_H(m \| r)$
- In $\mathsf{Game}_1$: $h$ is chosen uniformly at random, then we program the oracle so that $\mathcal{O}_H(m \| r) = h$
- In $\mathsf{Game}_1$: If $h'$ is a valid hash commitment (i.e., $\exists (m', r')$ such that $\mathcal{O}_H(m' \| r') = h'$), then $m \oplus m' = m^*$, where $m^*$ was determined before running $\mathcal{A}$.
:::
:::

:::{exercise}
:label: ex-prog-rom-game-f

Consider $\mathsf{Game}_F$ which is exactly like $\mathsf{Game}_1$ except that $m^* := 0$ (instead of sampling uniformly). Give a PPT adversary $\mathcal{A}$ that makes one random oracle query and achieves

$$
\left|\Pr[\mathsf{Game}_0 = 1] - \Pr[\mathsf{Game}_F = 1]\right| = \frac{1}{2}.
$$

:::{dropdown} **Solution**

Consider the following adversary $\mathcal{A}$:

*First invocation:*
- Query $h' \leftarrow \mathcal{O}_H(0 \| 0^\lambda)$
- Output $(\bot, h')$

*Second invocation:*
- Output $m$

Analysis:
- In $\mathsf{Game}_0$: $m$ is uniform, so $\Pr[\mathsf{Game}_0 = 1] = \frac{1}{2}$.
- In $\mathsf{Game}_F$: Since $m^* = 0$ (fixed in $\mathsf{Game}_F$) and $m' = 0$, we have $m = m^* \oplus m' = 0 \oplus 0 = 0$. Thus $\Pr[\mathsf{Game}_F = 1] = 0$.

Therefore: $\left|\Pr[\mathsf{Game}_0 = 1] - \Pr[\mathsf{Game}_F = 1]\right| = \left|\frac{1}{2} - 0\right| = \frac{1}{2}$.
:::
:::

:::{exercise}
:label: ex-prog-rom

Sketch a proof of Proposition {ref}`prop-prog-rom`.

*Hints:*
1. Let $E$ be the event that the adversary queries $m \| r$ to the oracle during its first invocation (i.e., the event that triggers the assertion). Argue that $\Pr[\mathsf{Game}_0 = 1 \wedge \neg E] = \Pr[\mathsf{Game}_1 = 1 \wedge \neg E]$.
2. Bound the probability of $E$.

:::{dropdown} **Solution**

Let $E$ be the event that the adversary queries $m \| r$ to the oracle during its first invocation (when it outputs $(\mathit{state}, h')$).

We claim that $\Pr[\mathsf{Game}_0 = 1 \wedge \neg E] = \Pr[\mathsf{Game}_1 = 1 \wedge \neg E]$. To see this, we analyze the distribution of all values conditioned on $\neg E$:

- *The value $h$:* In $\mathsf{Game}_0$, $h = \mathcal{O}_H(m \| r)$ where the oracle uses lazy sampling. Since $\neg E$ occurs, the adversary never queried $m \| r$ during the first invocation, so $h$ is a fresh uniform sample. In $\mathsf{Game}_1$, $h$ is explicitly drawn uniformly at random. Thus $h$ is uniform in $\{0,1\}^\lambda$ in both games.

- *The value $r$:* In both games, $r$ is drawn uniformly from $\{0,1\}^\lambda$.

- *The value $m$:* In $\mathsf{Game}_0$, $m$ is drawn uniformly from $\{0,1\}$. In $\mathsf{Game}_1$:
  - If $h'$ is a valid commitment: $m = m^* \oplus m'$ where $m^*$ is uniform in $\{0,1\}$ and independent of the adversary's view
  - If $h'$ is not a valid commitment: $m$ is drawn uniformly from $\{0,1\}$
  
  In both cases, $m$ is uniform in $\{0,1\}$.

- *The oracle programming:* In $\mathsf{Game}_1$, we set $T[m \| r] = h$. Since $\neg E$ occurs, the input $m \| r$ is fresh (never queried before), so this programming is invisible to the adversary and maintains consistency with $h = \mathcal{O}_H(m \| r)$.

Since all values have identical distributions and the oracle behaves identically in both games (returning the same values for the same queries), the adversary's view is bit-by-bit identical conditioned on $\neg E$. Therefore $\Pr[\mathsf{Game}_0 = 1 \wedge \neg E] = \Pr[\mathsf{Game}_1 = 1 \wedge \neg E]$.

By the difference lemma, we have:

$$
\left|\Pr[\mathsf{Game}_0 = 1] - \Pr[\mathsf{Game}_1 = 1]\right| \leq \Pr[E]
$$

To bound $\Pr[E]$, note that $E$ occurs if the adversary queries $m \| r$ during the first invocation. The key observation is that $r \in \{0,1\}^\lambda$ is chosen uniformly at random and independently of the adversary's first invocation:
- In $\mathsf{Game}_0$: $r$ is chosen at the beginning but is unknown to the adversary
- In $\mathsf{Game}_1$: $r$ is chosen after the first invocation

In both cases, for any specific query $x_i$ made by the adversary during the first invocation, the probability that $x_i = m \| r$ is at most $\frac{1}{2^\lambda}$ (the adversary must guess the $\lambda$-bit value $r$ correctly).

Since the adversary makes at most $q$ queries during the first invocation, by the union bound:

$$
\Pr[E] \leq \sum_{i=1}^q \Pr[x_i = m \| r] \leq \sum_{i=1}^q \frac{1}{2^\lambda} = \frac{q}{2^\lambda}
$$

Therefore:

$$
\left|\Pr[\mathsf{Game}_0 = 1] - \Pr[\mathsf{Game}_1 = 1]\right| \leq \frac{q}{2^\lambda}
$$
:::
:::

:::{exercise}
:label: ex-fork-no-query

Consider an algorithm $\mathcal{A}$ that never queries $\mathcal{O}_H$. What is $\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}}$?

:::{dropdown} **Solution**

If $\mathcal{A}$ never queries $\mathcal{O}_H$, then the table $T$ remains empty throughout the first execution. In the Fork game:

- First execution: $\mathcal{A}$ outputs $\alpha = (z, \mathit{aux})$ with probability $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}}$
- Since $z$ was never queried, $T[z] = \bot$ (undefined)
- We copy $T$ to $T'$, so $T'[z] = \bot$ as well
- We then set $T'[z] \xleftarrow{\$} S$ (a fresh random value)
- Second execution: Since $\mathcal{A}$ receives the same $\mathit{inp}$ and $\rho$, and never queries the oracle, it outputs the same $(z, \mathit{aux})$ deterministically
- The condition $z = z'$ is satisfied
- The condition $T[z] \neq T'[z]$ is satisfied since $T[z] = \bot$ while $T'[z] \in S$

Therefore:

$$
\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} = \mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}}
$$
:::
:::

:::{exercise}
:label: ex-fork-one-query

Consider an algorithm $\mathcal{A}$ that makes exactly one query $z \leftarrow \mathcal{O}_H(\rho)$ and outputs $(z, \bot)$. What is $\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}}$?

:::{dropdown} **Solution**

In this case, $\mathcal{A}$ always succeeds in outputting a non-$\bot$ value, so $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} = 1$. In the Fork game:

- First execution: $\mathcal{A}$ queries $z \leftarrow \mathcal{O}_H(\rho)$ and outputs $(z, \bot)$. Since $\rho$ is fixed, $z$ is determined by the oracle's response.
- We copy $T$ to $T'$, so $T'[\rho] = T[\rho] = z$
- We then set $T'[z] \xleftarrow{\$} S$ (refreshing the value at the forking point)
- Second execution: $\mathcal{A}$ queries $z' \leftarrow \mathcal{O}_{H'}(\rho) = T'[\rho] = z$ (unchanged) and outputs $(z', \bot) = (z, \bot)$
- The condition $z = z'$ is satisfied
- The condition $T[z] \neq T'[z]$ is satisfied with probability $1 - \frac{1}{|S|}$ (since $T'[z]$ was refreshed to a random value)

Therefore:

$$
\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} = 1 \cdot \left(1 - \frac{1}{|S|}\right) = 1 - \frac{1}{|S|}
$$

This matches the forking lemma bound when $q = 1$ and $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} = 1$:

$$
1 \cdot \left(\frac{1}{1} - \frac{1}{|S|}\right) = 1 - \frac{1}{|S|}
$$
:::
:::

:::{exercise}
:label: ex-fork-extraction

Let $\mathsf{InpGen}$ output $\mathit{inp} = (\mathbb{G},p,g)$ where $g$ is a generator of a cyclic group $\mathbb{G}$ of prime order $p \geq 2^{\lambda-1}$.
Let $\mathcal{A}$ be an algorithm that makes at most $q$ queries to a random oracle $\mathcal{O}_H: \{0,1\}^* \rightarrow \mathbb{Z}_p$ and always succeeds in outputting $(z, s)$ such that $s = z + \mathcal{O}_H(z) \cdot x \bmod p$ for some fixed $x \in \mathbb{Z}_p$.
Using the forking lemma, give a PPT algorithm $\mathcal{B}$ that outputs $x$ and bound its success probability.

:::{dropdown} **Solution**

We construct a PPT algorithm $\mathcal{B}$ that runs the forking game and extracts $x$ from the outputs.
If $\mathsf{Game~Fork}[\mathsf{InpGen}]$ returns $(z, s, z', s') \neq \bot$, then we have:
- From the first execution: $(z, s) = \alpha$ where $s = z + T[z] \cdot x \bmod p$
- From the second execution: $(z', s') = \alpha'$ where $s' = z' + T'[z'] \cdot x \bmod p$
- The conditions $z = z'$ and $T[z] \neq T'[z]$ hold

Therefore: $s = z + T[z] \cdot x$ and $s' = z + T'[z] \cdot x \pmod{p}$.

Subtracting yields $s - s' = (T[z] - T'[z]) \cdot x \pmod{p}$. Since $T[z] \neq T'[z]$ and $p$ is prime, we can compute:

$$
x = (s - s') \cdot (T[z] - T'[z])^{-1} \bmod p
$$

Since $\mathcal{A}$ always succeeds, $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} = 1$. By the forking lemma (noting that $S = \mathbb{Z}_p$ where $|S| = p \geq 2^{\lambda-1}$):

$$
\Pr[\mathcal{B} \text{ outputs } x] = \mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} \geq 1 \cdot \left(\frac{1}{q} - \frac{1}{|S|}\right) \geq \frac{1}{q} - \frac{1}{2^{\lambda-1}}
$$
:::
:::

:::{exercise}
:label: ex-fork-conditional

Consider an algorithm $\mathcal{A}$ that queries $h_0 \leftarrow \mathcal{O}_H(0)$ and outputs $(0, \bot)$ if $h_0 \bmod 2 = 0$, otherwise outputs $(h_0, \bot)$. What is $\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}}$?

:::{dropdown} **Solution**

First, note that $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} = 1$ since $\mathcal{A}$ always outputs a non-$\bot$ value.

The adversary only queries $\mathcal{O}_H(0)$, so after the first execution, $T[0] = h_0$ and all other entries remain $\bot$.

**Case 1:** $h_0 \bmod 2 = 0$ (happens with probability $\frac{1}{2}$)
- First execution: $\mathcal{A}$ outputs $(0, \bot)$
- We set $T'[0] \xleftarrow{\$} S$ (refreshing the value at key 0)
- Second execution: $\mathcal{A}$ queries $h_0' = T'[0]$ (the refreshed value)
  - If $h_0' \bmod 2 = 0$: outputs $(0, \bot)$, so $z = z' = 0$ and $T[0] = h_0 \neq h_0' = T'[0]$ with probability $1 - \frac{1}{|S|}$
  - If $h_0' \bmod 2 = 1$: outputs $(h_0', \bot)$, so $z = 0 \neq h_0' = z'$ (fork fails)
- Fork succeeds with probability $\frac{1}{2} \cdot (1 - \frac{1}{|S|})$

**Case 2:** $h_0 \bmod 2 = 1$ (happens with probability $\frac{1}{2}$)
- First execution: $\mathcal{A}$ outputs $(h_0, \bot)$
- Since $h_0$ was never queried, $T[h_0] = \bot$
- We set $T'[h_0] \xleftarrow{\$} S$
- Second execution: $\mathcal{A}$ queries $h_0' = T'[0] = T[0] = h_0$ (unchanged)
- Since $h_0 \bmod 2 = 1$, outputs $(h_0, \bot)$
- So $z = z' = h_0$ and $T[h_0] = \bot \neq T'[h_0]$
- Fork succeeds with probability 1

Therefore:

$$
\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}} = \frac{1}{2} \cdot \frac{1}{2} \cdot \left(1 - \frac{1}{|S|}\right) + \frac{1}{2} \cdot 1 = \frac{1}{4} \left(1 - \frac{1}{|S|}\right) + \frac{1}{2} \approx \frac{3}{4}
$$
:::
:::

:::{exercise}
:label: ex-fork-tight

**(Optional)** Find an algorithm $\mathcal{A}$ making exactly $q$ queries for which $\mathsf{Adv}^{\mathsf{Fork}}_{\mathcal{A}, \mathsf{InpGen}}$ is as close to $\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}} \left( \frac{\mathsf{Adv}^{\mathsf{Single}}_{\mathcal{A}, \mathsf{InpGen}}}{q} - \frac{1}{|S|} \right)$ as possible.
:::

:::{exercise}
:label: ex-fork-comparison

**(Optional)** Compare the Generalized Forking Lemma of Bellare and Neven (CCS 2006) with the Local Forking Lemma. What are the key differences in their formulations and applications?
:::